In [ ]:
# Imports
from mltools import mlproc as mlp
from mltools import mlstudy as mls
from mltools import mdaio
import os

In [ ]:
# Synthesize timeseries
duration=600
M=4
samplerate=30000
mlp.runProcess('ephys.synthesize_random_waveforms',{},{'geometry_out':'geom.csv','waveforms_out':'waveforms_true.mda'},{'upsamplefac':13,'M':M,'average_peak_amplitude':100})
mlp.runProcess('ephys.synthesize_random_firings',{},{'firings_out':'firings_true.mda'},{'duration':duration})
mlp.runProcess('ephys.synthesize_timeseries',{'firings':'firings_true.mda','waveforms':'waveforms_true.mda'},{'timeseries_out':'raw.mda.prv'},{'duration':duration,'waveform_upsamplefac':13,'noise_level':10})

In [ ]:
# View the timeseries
os.system('ev-view-timeseries raw.mda.prv --firings firings_true.mda')

In [ ]:
# Preprocessing
freq_min=300
freq_max=6000
mlp.runProcess('ephys.bandpass_filter',{'timeseries':'raw.mda.prv'},{'timeseries_out':'filt.mda.prv'},{'samplerate':samplerate,'freq_min':freq_min,'freq_max':freq_max})
mlp.runProcess('ephys.whiten',{'timeseries':'filt.mda.prv'},{'timeseries_out':'pre.mda.prv'},{})

In [ ]:
# Sorting
pp={}
pp['detect_sign']=1
pp['adjacency_radius']=-1
pp['detect_threshold']=3
mlp.runProcess('ms4alg.sort',{'timeseries':'pre.mda.prv','geom':'geom.csv'},{'firings_out':'firings.mda'},pp)

In [ ]:
# Compute templates
templates=mlp.runProcess('ephys.compute_templates',{'timeseries':'filt.mda.prv','firings':'firings.mda'},{'templates_out':True},{'clip_size':150})['templates_out']
os.system('ev-view-templates {}'.format(templates))